# Four point functions in the $O(n)$ model

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [3]:
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BarnesDoubleGamma, BenchmarkTools
println("Number of threads: ", Threads.nthreads())

Number of threads: 4


# Bootstrap equations

We solve crossing symmetry equations for the spectrum of the $O(n)$ CFT:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
\end{align}
We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(1-x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(1/x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

In [4]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c = CC(β=1 / (big"0.8" + big"0.2" * im))
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
P = big"0.2" + big"0.1" * im
diag_field = Field(c, P=P)
fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], Field(c, P=P))

# Determine the parity of the number of legs in 4pt channels
function chan_parities(co::Correlation4)
    V1, V2, V3, V4 = co.fields
    chan_parities = Channels{Rational}((V1.r + V2.r) % 1, (V1.r + V4.r) % 1, (V1.r + V3.r) % 1)
end

# Compute series of blocks for any channel field compatible
# with the correlation co
function LoopSpectra(co, fields, fs)
    Vs = @channels filter(V -> V.r % 1 == chan_parities(co)[chan], fields)
    @channels ChannelSpectrum(co, chan, Vs[chan], fs[chan])
end

# Compute series of blocks, keeping only one of each (field, reflected field) pair
# given indices of the external fields.
function precompute_blocks(
    indices, fields=nothing;
    c=CC(β=1 / (big"0.8" + big"0.2" * im)),
    P=big"0.2" + big"0.1" * im,
    parity, precision=10
)
    setprecision(BigFloat, floor(Int, 1.2 * precision), base=10)
    Δmax = floor(Int, 1.5 * precision)
    if fields === nothing
        ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
        fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], Field(c, P=P))
    end
    co = Correlation([Field(c, r=r, s=s) for (r, s) in indices], Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    fs = Channels(chan -> (V -> IBlock(co, chan, V, parity=parity)))
    LoopSpectra(co, fields, fs)
end

function compute_diagblocks(specs, Ps)
    co = specs.s.corr
    c = co.fields[1].c
    return @channels [IBlock(co, chan, Field(c, P=P)) for P in Ps[chan]]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
    specs, signature;
    even_spin=Channels(false),
    rmax=3, show=true, Ps=nothing,
    fix=nothing,
    rels=nothing,
)
    specs = @channels filter(V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan], specs[chan])
    specs = @channels filter(V -> even_spin[chan] ? spin(V) % 2 == 0 : true, specs[chan])
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix, rels=rels)
        if show
            if rels === :stu
                printstyled("Channels s, t, u:\n", bold=true)
                Base.show(stdout, res.str_cst.s, rmax=rmax)
            else
                Base.show(stdout, res.str_cst, rmax=rmax)
            end
        end
    else
        diagblocks = compute_diagblocks(specs, Ps)
        res = solve_bootstrap(specs, diagblocks, fix=fix, rels=rels)
    end
    return res
end;

# Some numerical four-point functions

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [5]:
ind = ((1//2, 0), (1//2, 0), (1//2, 0), (1//2, 0))
blocks = precompute_blocks(ind, fields, parity=0, precision=20);

### Signature $(0, 1, 1)$

In [6]:
sol = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false), rmax=2);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ P = (0.2+0.1im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ -0.167173+0.134319im      │ 4.2e-18   │
│ (2, 0)          │ -0.00239852+0.000773346im │ 1.8e-16   │
│ (2, 1)          │ 0.000797319-0.000705013im │ 1.2e-16   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌────────────┬───────────────────────────┬───────────┐
│ Field      │ Structure constant        │ Rel. err. │
├────────────┼───────────────────────────┼───────────┤
│ (1, 0)     │ 0.422102+0.0893706im      │ 1.6e-18   │
│ (1, 1)     │ 0.16568-0.0434621im       │ 6.9e-19   │
│ (2, 0)     │ 0.000999679-1.2823e-05im  │ 3.9e-16   │
│ (2, -1//2) │ 0.000592214+0.000185033im │ 8.3e-16   │
│ (2, 1//2)  │ 0.000592214+0.000185033im │ 9e-16     │
│ (2, 1)     │ 0.000344058+0.000230614im │ 6e-16     │
└──

In [ ]:
# latex output renders in the notebook
display(c)
display(sol.correlation)
# latex copy-pastable output with backend=:latex keyword argument:
show(stdout, sol.str_cst, rmax=2, backend=:latex)

c = 1.614532871972318339100346020761245674740484429065743944636678200692041522491287 + 2.232249134948096885813148788927335640138408304498269896193771626297577854671372im, β = -1.176470588235294117647058823529411764705882352941176470588235294117647058823537 + 0.2941176470588235294117647058823529411764705882352941176470588235294117647058841im

NonChiralCorrelation{Complex{BigFloat}} with external fields
< (1//2, 0) (1//2, 0) (1//2, 0) (1//2, 0) >

 Channel s
\begin{tabular}{lll}
  \hline
  \textbf{Field} & \textbf{Structure constant} & \textbf{Relative error} \\\hline
  $(P=0.10+0.10im)$ & $1.0$ & 0.0 \\
  $\left(1,0\right)$ & $-0.20949593\;+\;0.113615565\mathrm{i}$ & 8.03089e-18 \\
  $\left(2,0\right)$ & $-0.0030127857\;+\;0.0008712257\mathrm{i}$ & 3.86131e-16 \\
  $\left(2,1\right)$ & $0.0010190255\;-\;0.0008476949\mathrm{i}$ & 1.99533e-16 \\\hline
\end{tabular}
\\ Channel t
\begin{tabular}{lll}
  \hline
  \textbf{Field} & \textbf{Structure constant} & \textbf{Relative error} \\\hline
  $\left(1,0\right)$ & $0.5220995\;+\;0.12710619\mathrm{i}$ & 2.45324e-18 \\
  $\left(1,1\right)$ & $0.2078798\;-\;0.04788174\mathrm{i}$ & 1.16366e-18 \\
  $\left(2,0\right)$ & $-0.0011053841\;+\;0.0008902788\mathrm{i}$ & 5.09191e-16 \\
  $\left(2,\frac{-1}{2}\right)$ & $-0.0010755917\;+\;0.0011621625\mathrm{i}$ & 6.73923e-16 \\
  $\left(2,\frac{1}{2}\right)$ & $-0.0010755917\;+\;0.0011621625\mathrm{i}$ & 5.25469e-16 \\
  $\left(2,1\right)$ & $-0

### Signature $(0, 1, 0)$, $D^{(s)}_{(2, 0)} = 0$

In [5]:
fix = [(:s, diag_field, 1), (:s, Field(c, r=2, s=0), 0)]
solve(blocks, Sig(0, 1, 0), fix=fix, rmax=2);

 Channel s
┌─────────────────┬──────────────────────────┬────────────────┐
│ Field           │ Structure constant       │ Relative error │
├─────────────────┼──────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                1.0+0.0im │            0.0 │
│          (1, 0) │      -1.16517-0.591022im │    7.65818e-15 │
│          (1, 1) │     0.464566+0.0798147im │     8.4386e-15 │
│          (2, 0) │                0.0+0.0im │            0.0 │
│       (2, 1//2) │ -0.00318409+0.00145081im │    1.01034e-14 │
│      (2, -1//2) │ -0.00318409+0.00145081im │     8.2106e-15 │
│          (2, 1) │  0.00257332-0.00274978im │    6.86828e-15 │
└─────────────────┴──────────────────────────┴────────────────┘
 Channel t
┌────────────┬──────────────────────────┬────────────────┐
│ Field      │ Structure constant       │ Relative error │
├────────────┼──────────────────────────┼────────────────┤
│     (1, 0) │     -0.433576-0.577531im │    1.38597e-14 │
│     (1, 1) │     0.672446+0.0319329i

### Solve for many different Ps

In [78]:
# solve for many different values of the momentum for the diag field in the channel.
# only recompute one column of the linear system each time: very cheap.
Ps = [big"0.8" + big"0.2"*im + i / big(100)*im for i in 1:3];
diagblocks = compute_diagblocks(blocks, Ps, :s)
sol_Ps = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),  Ps=Ps, diagchan=:s)
# print one of the results
show(stdout, sol_Ps[2].str_cst, rmax=2)

InterchiralBlock{Complex{BigFloat}}[G^(s)({ V_{P = 0.80000000000000000000000017 + 0.21000000000000000000000003im + n/β}, n ∈ -5:1:3}), G^(s)({ V_{P = 0.80000000000000000000000017 + 0.22000000000000000000000002im + n/β}, n ∈ -5:1:4}), G^(s)({ V_{P = 0.80000000000000000000000017 + 0.23000000000000000000000002im + n/β}, n ∈ -5:1:4})]
 Channel s
┌─────────────────┬─────────────────────────┬───────────┐
│ Field           │ Structure constant      │ Rel. err. │
├─────────────────┼─────────────────────────┼───────────┤
│ (1, 0)          │ 0.253897-0.354829im     │ 5.3e-18   │
│ (P=0.80+0.22im) │ 1.0+0.0im               │ 0         │
│ (2, 0)          │ 0.00488132-0.00418444im │ 2.1e-16   │
│ (2, 1)          │ -0.00118272+0.0024477im │ 1.2e-16   │
└─────────────────┴─────────────────────────┴───────────┘
 Channel t
┌────────────┬─────────────────────────┬───────────┐
│ Field      │ Structure constant      │ Rel. err. │
├────────────┼─────────────────────────┼───────────┤
│ (1, 0)     │ -1.0826

Benchmarks: on my 2015, 4-core Intel Macbook pro.
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

## $\left\langle P_1 P_2 P_3 P_4 \right\rangle$

In [7]:
ss = rand(Complex{BigFloat}, 4)
ind = Tuple((0, s) for s in ss)
blocks = precompute_blocks(ind, fields, parity=1, precision=15);

In [8]:
sol = solve(blocks, Sig(0, 0, 0), show=true);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │      -0.043652+0.0461863im │    6.68646e-10 │
│          (1, 1) │     -0.00502903+0.098547im │     1.1386e-10 │
│          (2, 0) │   0.00079105-0.000735744im │     4.28782e-8 │
│       (2, 1//2) │  0.000110591-0.000183974im │     1.46143e-7 │
│          (2, 1) │ -0.000189192-3.69278e-06im │     5.65625e-8 │
│          (3, 0) │                          0 │      0.0607838 │
│       (3, 1//3) │                          0 │      0.0644803 │
│       (3, 2//3) │                          0 │      0.0288652 │
│          (3, 1) │                          0 │      0.0139439 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬───────────────────────────┬───────

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [9]:
ind = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))
blocks = precompute_blocks(ind, fields2, parity=1, precision=13);

UndefVarError: UndefVarError: `fields2` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Signature $(0, \frac32, \frac32)$

In [10]:
sol = solve(blocks, Sig(0, 3//2, 3//2));

 Channel s
┌─────────────────┬────────────────────────┬────────────────┐
│ Field           │ Structure constant     │ Relative error │
├─────────────────┼────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │              1.0+0.0im │            0.0 │
│          (1, 0) │    -1.79924+0.421555im │      0.0536187 │
│          (1, 1) │ 0.000273845+0.203333im │       0.179268 │
│          (2, 0) │      1.05837-2.37073im │       0.276612 │
│       (2, 1//2) │   -0.557097+0.386062im │        1.79628 │
│          (2, 1) │    -0.133632-0.76917im │       0.431461 │
│          (3, 0) │      41.9787-27.2899im │       0.626559 │
│       (3, 1//3) │     -34.9665+26.8837im │       0.680484 │
│       (3, 2//3) │       24.938-25.2727im │       0.667133 │
│          (3, 1) │     -11.1594+19.0543im │       0.716201 │
└─────────────────┴────────────────────────┴────────────────┘
 Channel t
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├─

### Signature $(1, \frac12, \frac32)$

In [11]:
solve(blocks, Sig(1, 1//2, 3//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.314676-0.227065im │        1.58253 │
│    (2, 0) │   -0.756645+5.0774im │       0.595367 │
│ (2, 1//2) │  -0.515941-3.00656im │       0.911785 │
│    (2, 1) │   0.973783+1.64767im │        1.26323 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (1, 0) │ -0.0288501+0.0165005im │          1.011 │
│    (1, 1) │  0.0329273-0.0770832im │       0.777737 │
│    (2, 0) │      3.39492+1.71516im │          2.017 │
│ (2, 1//2) │     -2.78334-1.07935im │        1.75393 │
│    (2, 1) │    1.69856+0.0715347im │        1.71933 │
└───────────┴────────────────────────┴──────

### Signature $(1, \frac32, \frac12)$

In [12]:
solve(blocks, Sig(1, 3//2, 1//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.422351-0.268128im │       0.215619 │
│    (2, 0) │    -2.42269+4.0059im │       0.383119 │
│ (2, 1//2) │    1.60603-2.13824im │       0.575963 │
│    (2, 1) │  -0.166614+1.56393im │       0.488147 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────┬────────────────┐
│ Field     │ Structure constant │ Relative error │
├───────────┼────────────────────┼────────────────┤
│    (2, 0) │  2.02455+1.94047im │       0.673465 │
│ (2, 1//2) │ -1.66763-1.39314im │       0.676533 │
│    (2, 1) │ 1.12007+0.374722im │       0.710549 │
└───────────┴────────────────────┴────────────────┘
 Channel u
┌───────────┬───────────────────────┬────────────────┐
│ Field     │ Structure constant    │ Relative error │
├──────

## $\left\langle (0, \frac12)^3 P_{(1, 0)} \right\rangle$

In [3]:
ind = ((0, 1//2), (0, c.β^2), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
sol_diag = solve(blocks, Sig(0, 0, 0), even_spin=Channels(true), rmax=3);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ 1.35314+0.621346im        │ 3.2e-18   │
│ (2, 0)          │ -0.0146027+0.00422597im   │ 1.9e-16   │
│ (2, 1)          │ -9.09864e-05+0.00247326im │ 1.3e-16   │
│ (3, 0)          │ 1.58764e-06+1.66872e-06im │ 1.9e-12   │
│ (3, 2//3)       │ 1.18639e-06-4.9787e-07im  │ 1.8e-12   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0-8.82188e-20im         │ 3.4e-19   │
│ (1, 0)          │ 1.35314+0.621346im        │ 3e-18     │
│ (2, 0)          │ -0.0146027+0.00422597im   │ 9.6e-17   │
│ (2, 1)          

## $\left\langle (0, \frac12)^3 (1, 0) \right\rangle$

In [4]:
ind = ((0, 1//2), (1, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);

In [5]:
sol1 = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),show=true, rmax=2);
sol2 = solve(blocks, Sig(1, 0, 1), even_spin=Channels(false, true, false), show=false);
sol3 = solve(blocks, Sig(1, 1, 0), even_spin=Channels(false, false, true), show=false);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ 0                         │ 0.31      │
│ (2, 0)          │ -0.00585115-0.000436412im │ 1.5e-17   │
│ (2, 1)          │ 0.00227685-0.000981606im  │ 1.5e-17   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌───────────┬──────────────────────────┬───────────┐
│ Field     │ Structure constant       │ Rel. err. │
├───────────┼──────────────────────────┼───────────┤
│ (1, 0)    │ 0.676571+0.310673im      │ 8.6e-20   │
│ (1, 1)    │ 0.286591-0.0446828im     │ 4.8e-20   │
│ (2, 0)    │ -0.00437577+0.00233119im │ 1e-17     │
│ (2, 1//2) │ -0.00327768+0.00228679im │ 9.9e-18   │
│ (2, 1)    │ -0.00118392+0.00172743im │ 8.3e-18   │
└───────────┴──────────────────────────┴───────────┘
 Channel u
┌─────────

In [6]:
show(
    stdout,
    sol1.str_cst.s + sol2.str_cst.s + sol3.str_cst.s - sol_diag.str_cst.s,
    rmax = 2
)

┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0                          │ 0.31      │
│ (1, 1)          │ 5.17633e-20-1.52054e-19im  │ 6e-19     │
│ (2, 0)          │ -2.60874e-19-4.11794e-18im │ 4.3e-16   │
│ (2, 1//2)       │ 1.18137e-18+1.02885e-18im  │ 2.7e-16   │
│ (2, 1)          │ -6.71868e-19-5.58104e-19im │ 4e-16     │
└─────────────────┴────────────────────────────┴───────────┘


## $\left\langle (0, \frac12)^3 (2, 0) \right\rangle$

In [16]:
ind = ((0, 1//2), (2, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
solve(blocks, Sig(1, 2, 1), even_spin=Channels(false, true, false), rmax=3);

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │        -0.63854+0.181026im │    2.98082e-17 │
│    (2, 0) │                          0 │       0.880898 │
│ (2, 1//2) │                          0 │       0.970237 │
│    (2, 1) │                          0 │        1.06903 │
│    (3, 0) │ -7.86521e-05-0.000322813im │    9.03894e-12 │
│ (3, 1//3) │   4.37238e-05+0.00014278im │    1.86741e-11 │
│ (3, 2//3) │  4.92031e-05+9.64753e-05im │    1.78644e-11 │
│    (3, 1) │ -8.77471e-05-9.14424e-05im │    9.44656e-12 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (2, 0) │    0.413186+0.31

## $\left\langle (0, \frac12)^3 (3, 0) \right\rangle$

# Polynomials

In [18]:
import BootstrapVirasoro.LoopModels: compute_reference, ρ_residue

wrs(r, s, β) = 2cospi(r*β^2-s)

βs = [1 / (big"0.8" + big"0.2" * im) + (i-1) // 10 for i in 1:3]
cs = [CC(β=β) for β in βs]
inds = [((0, 1//2), (0, β^2), (0, 1//2), (0, 1//2)) for β in βs]
Ps = Channels([big"0.2" + big"0.2" * im + i // 100 * im for i in 1:5]) # diag field identical in each channel
blocks = [[precompute_blocks(inds[j], c=c, P=Ps[:s][i], parity=1, precision=20) for i = eachindex(Ps[:s])] for (j, c) in enumerate(cs)]
sols = [[solve(blocks[j][i], Sig(0, 0, 0), even_spin=Channels(true), show=false, rels=:stu) for i = eachindex(Ps[:s])] for j = eachindex(cs)];

In [23]:
r, s = (2, 0)
vals = [[sols[j][i].str_cst.s.constants[Field(c, r=r, s=s)] for i = eachindex(Ps[:s])] for j = eachindex(cs)]
for (j, c) = enumerate(cs)
    DG = DoubleGamma(c.β)
    V = Field(c, r=r, s=s)
    for (i, P) in enumerate(Ps[:s])
        co = sols[j][i].correlation
        diag_field = Field(c, P=P)
        w = 2cospi(2c.β * P)
        DP = compute_reference(co, diag_field, :s, DG)
        DV = compute_reference(co, V, :s, DG)
        vals[j][i] *= DP / DV
        vals[j][i] -= ρ_residue(V, co.fields...) / (w - wrs(V.r, V.s, c.β))
    end
end
data = [((2cospi(2c.β * P), -2cospi(c.β^2)), vals[j][i]) for (i, P) in enumerate(Ps[:s]) for (j, c) in enumerate(cs)];

In [ ]:
j = 1
da = [((data[j][i][1][1],), data[j][i][2]) for i in eachindex(data)]
fit!(Polyfit((:w, ), (2, )), da)

BoundsError: BoundsError: attempt to access Tuple{Tuple{Complex{BigFloat}, Complex{BigFloat}}, Complex{BigFloat}} at index [3]

In [176]:
loopn(c) = -2cospi(c.β^2)
loopw(c, P) = 2cospi(2*c.β*P)
ccs = [CC(β=β) for β in βs]
perfect_data = [((loopw(c, P), loopn(c)), (loopn(c)^2-4) * (loopw(c, P)^2 + loopw(c, P) - 2)) for (i, P) in enumerate(Ps[:s]) for (j, c) in enumerate(ccs)]
pf = Polyfit((:w, :n), (2, 2))
fit!(pf, perfect_data)

Polyfit((:w, :n), (2, 2), 9, [(0, 0) (0, 1) (0, 2); (1, 0) (1, 1) (1, 2); (2, 0) (2, 1) (2, 2)], Complex{BigFloat}[8.0000000000000000000088806 + 2.2662553721317715301588485e-21im, -3.9999999999999999999993085 - 2.7503423110654502742604003e-21im, -4.0000000000000000000001985 - 4.9263177880473411963421303e-23im, -1.8618388085914526466692239e-21 - 1.8020375211632901946383187e-21im, -8.1066989468176145598363137e-22 + 2.0817019705221386162543302e-22im, -1.8869287817217856872369095e-23 + 8.1231014818516334517822148e-23im, -2.0000000000000000000000629 + 2.5958278282264909073408701e-22im, 1.000000000000000000000086 + 1.1110512255951053126051484e-22im, 1.0000000000000000000000199 - 6.4724906440693780939062645e-25im])

In [183]:
data

35-element Vector{Tuple{Tuple{Complex{BigFloat}, Complex{BigFloat}}, Complex{BigFloat}}}:
 ((-1.0401664175278576877921979 - 2.8284728346181571760788606im, 5.2907289158617245824455991 + 6.9837792049850925824198603im), 268.49087557263867748792024 - 257.24096630002868382435212im)
 ((-0.64718823176235127953390552 - 3.3840482239262533644760169im, 2.0591161221952358385353872 + 3.5088540129207270879381092im), 87.4544762507077419917468 - 18.064801312632058365036309im)
 ((-0.1409125825822382270976279 - 3.9552929258957942550827329im, 0.95655155564420883896672913 + 1.3921641124881092995857899im), 35.135518128182484356776794 + 3.6142251261239306009541018im)
 ((0.49673316146867298467769953 - 4.5244456305756391279681875im, 0.71321306951618212614103037 - 0.081283776129569926670955538im), 25.950621545386528773628715 + 13.76990915701083747824797im)
 ((1.2835276301622262615959689 - 5.069393315724418669773239im, 1.0283198772407456614575342 - 1.5939868946338840440147648im), 13.993384734611570411271821 + 5

In [180]:
[perfect_data[i][2] for i in eachindex(data)]

35-element Vector{Complex{BigFloat}}:
  20.973383857826160397113485 - 811.64149976300077081856674im
  150.75257973296248823292503 - 209.70750925326043101697913im
  96.803406326627403823101121 - 33.04687358798754004488202im
  74.954325927012407922820794 + 34.068181393121530372442139im
  76.530372668390787424953453 + 180.34953542342470095016812im
 -27.564361149218124096957472 - 932.18414490210276586648105im
  163.91525932945311839014753 - 248.28550397219823498754022im
  110.71321478582185277342073 - 40.943747794397915267845512im
  87.022917291022933374751161 + 38.374896126228889758825769im
   87.94871084122576292702594 + 208.6462645317447636970469im
                              ⋮
  220.06685423997211230429595 - 480.86722033482468909643931im
  189.53967562408827668759809 - 91.57235950043963469550122im
  158.04003878534309883816429 + 62.330284887091884114284349im
  151.47639271353905802276023 + 375.26190797893976222911796im
 -561.33466605675154004278051 - 1822.8014718261904075986437im
  2